In [2]:
!pip install ray


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 11.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

# Check your last digit of the roll number for dataset selection
last_digit = 207  # Replace this with your last digit
if last_digit % 3 == 0:
    dataset_name = "stl10"
elif last_digit % 3 == 1:
    dataset_name = "svhn"
else:
    dataset_name = "fashionmnist"

# Define transform based on the dataset
if dataset_name == "stl10":
    data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    num_classes = 10
elif dataset_name == "svhn":
    data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    num_classes = 10
else:  # FashionMNIST
    data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    num_classes = 10

# Load dataset
if dataset_name == "stl10":
    train_data = datasets.STL10(root="./data", split='train', download=True, transform=data_transforms)
    test_data = datasets.STL10(root="./data", split='test', download=True, transform=data_transforms)
elif dataset_name == "svhn":
    train_data = datasets.SVHN(root="./data", split='train', download=True, transform=data_transforms)
    test_data = datasets.SVHN(root="./data", split='test', download=True, transform=data_transforms)
else:  # FashionMNIST
    train_data = datasets.FashionMNIST(root="./data", train=True, download=True, transform=data_transforms)
    test_data = datasets.FashionMNIST(root="./data", train=False, download=True, transform=data_transforms)

# Define data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Define training function
def train_cnn(config):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer_name = config["optimizer"]
    if optimizer_name == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9)
    elif optimizer_name == "adam":
        optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    for epoch in range(5):  # Adjust as needed
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100. * correct / total

        tune.report(loss=train_loss, accuracy=train_accuracy)

# Hyperparameter search space
search_space = {
    "lr": tune.loguniform(1e-6, 1e-1),
    "optimizer": tune.choice(["sgd", "adam"])
}

# Initialize Ray Tune
scheduler = ASHAScheduler(
    max_t=10,
    grace_period=1,
    reduction_factor=2,
    metric="loss",  # Specify the metric to optimize
    mode="min"      # Specify the optimization mode (minimize the loss)
)
reporter = CLIReporter(
    metric_columns=["loss", "accuracy", "training_iteration"])
analysis = tune.run(
    train_cnn,
    config=search_space,
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=reporter)

# Get best hyperparameters
best_config = analysis.get_best_config(metric="loss", mode="min")
best_lr = best_config["lr"]
best_optimizer = best_config["optimizer"]

# Fine-tune the model with the best hyperparameters
best_model = models.resnet18(pretrained=True)
num_ftrs = best_model.fc.in_features
best_model.fc = nn.Linear(num_ftrs, num_classes)

if best_optimizer == "sgd":
    optimizer = optim.SGD(best_model.parameters(), lr=best_lr, momentum=0.9)
elif best_optimizer == "adam":
    optimizer = optim.Adam(best_model.parameters(), lr=best_lr)

best_model.to("cuda" if torch.cuda.is_available() else "cpu")
best_model.train()

for epoch in range(5):  # Adjust as needed
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = best_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Accuracy: {100.*correct/total}")

# Test the model
best_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = best_model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = 100. * correct / total
print(f"Final top-5 test accuracy: {test_accuracy}%")


Files already downloaded and verified
Files already downloaded and verified


2024-03-31 10:31:42,060	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 2. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2024-03-31 10:31:43,890	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2024-03-31 10:31:45,876	WARNING tune.py:912 -- AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


Streaming output truncated to the last 5000 lines.
| train_cnn_de74a_00009   PENDING    0.000503754   adam        |
+--------------------------------------------------------------+
Trial status: 10 PENDING
Current time: 2024-03-31 11:36:33. Total running time: 1hr 4min 47s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------------+
| Trial name              status              lr   optimizer   |
+--------------------------------------------------------------+
| train_cnn_de74a_00000   PENDING    2.99112e-05   adam        |
| train_cnn_de74a_00001   PENDING    0.00347047    sgd         |
| train_cnn_de74a_00002   PENDING    0.00279636    adam        |
| train_cnn_de74a_00003   PENDING    7.00911e-06   sgd         |
| train_cnn_de74a_00004   PENDING    2.1388e-06    adam        |
| train_cnn_de74a_00005   PENDING    0.00115978    adam        |
| train_cnn_de74a_00006   PENDING    6.79234e-05   sgd         |
| t